---
layout : post
title : Sprint 5
descriptions : Sprint 5
courses : { csp: {week: 1} }
comments: true
sticky_rank: 1

---


<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>National Park Review Platform</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #e8f5e9;
            color: #2e3b4e;
            }

</style>
</head>
<body>
    <div class="container">
 <h4>National Park Review Platform</h4>
        <p>Our platform is designed to enhance outdoor adventure experiences by connecting users with national parks and providing tools to make their trips unforgettable. The platform offers several key features:</p>
        
<h5>Platform Features</h5>
<ul>
            <li><strong>Personality Quiz:</strong> Matches users with a national park based on their unique traits and preferences.</li>
            <li><strong>Tailored Camping Tips:</strong> Offers camping advice specific to the assigned park, along with a chatroom for discussions and shared experiences.</li>
            <li><strong>Park Information:</strong> Directs users to the official national park webpage to learn more about their assigned destination.</li>
            <li><strong>Ratings and Reviews:</strong> Enables users to provide feedback about their experiences after visiting a park.</li>
        </ul>
<div class="contribution">
            <h5>My Contribution</h5>
            <p>I developed a comprehensive <strong>analytics page</strong> to enhance the platform’s functionality. This page includes:</p>
            <ul>
                <li>A feature to display the <strong>number of stars</strong> a user has rated for different parks.</li>
                <li>Functionality to track and show <strong>which specific parks</strong> the ratings were given for.</li>

</ul>
        </div>
    </div>
</body>
</html>

-------------------------------------------------------------------------------------------------------------------------------------------------------


Postman connection running successfully

<div style="text-align: center; margin: 20px 0;">
  <img src="{{site.baseurl}}/images/postman.png" alt="Me at the student panel" style="width: 70%; max-width: 500px; border-radius: 15px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); animation: fadeIn 2s ease-in-out;" />
</div>

When running -init-, db_init, restore, and backup the tables remain constant 

<div style="text-align: center; margin: 20px 0;">
  <img src="{{site.baseurl}}/images/analytics.png" alt="Me at the student panel" style="width: 70%; max-width: 500px; border-radius: 15px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); animation: fadeIn 2s ease-in-out;" />
</div>


CRUD methods for my analytics model : 


    def create(self):
        db.session.add(self)
        db.session.commit()

    def read(self):
        return {
            'id': int(self.id),  # Explicitly converting to JSON-serializable types
            'channel_id': int(self.channel_id) if self.channel_id else None,
            'user_id': int(self.user_id),
            'stars': int(self.stars),
        }
        
    def update(self):
       
        db.session.add(self)
        db.session.commit()

    def delete(self):
       
        db.session.delete(self)
        db.session.commit()
        





Code Requests: 


    def post(self):
            """
            Create a new analytics entry.
            """
            data = request.get_json()

            # Validate input
            required_fields = ['channel_id', 'user_id', 'stars']
            if not all(field in data for field in required_fields):
                return {'message': 'Missing required fields'}, 400

            # Create and save a new analytics entry
            try:
                analytics = Analytics(
                    channel_id=data['channel_id'],
                    user_id=data['user_id'],
                    stars=data['stars']
                )
                analytics.create()
                return analytics.read(), 201
            except Exception as e:
                return {'message': f'An error occurred: {str(e)}'}, 500

        def get(self):
            """
            Retrieve analytics filtered by channel_id.
            """
            channel_id = request.args.get('channel_id')

            if not channel_id:
                return {'message': 'channel_id parameter is required'}, 400

            try:
                analytics_list = Analytics.query.filter_by(channel_id=channel_id).all()
                if not analytics_list:
                    return {'message': 'No analytics found for the given channel_id'}, 404

                return jsonify([entry.read() for entry in analytics_list]), 200
            except Exception as e:
                return {'message': f'An error occurred: {str(e)}'}, 500

    class _BULK_CRUD(Resource):
        def post(self):
            """
            Bulk create analytics entries.
            """
            data_list = request.get_json()

            if not isinstance(data_list, list):
                return {'message': 'Expected a list of analytics data'}, 400

            results = {'success': 0, 'errors': []}

            for data in data_list:
                try:
                    # Validate input
                    required_fields = ['channel_id', 'user_id', 'stars']
                    if not all(field in data for field in required_fields):
                        raise ValueError('Missing required fields')

                    # Create and save analytics entry
                    analytics = Analytics(
                        channel_id=data['channel_id'],
                        user_id=data['user_id'],
                        stars=data['stars'],
                    )
                    analytics.create() 
                    results['success'] += 1
                except Exception as e:
                    results['errors'].append({'data': data, 'error': str(e)})

            return jsonify(results), 207

    class _SUMMARY(Resource):
        def get(self):
            """
            Retrieve overall analytics summary data (reviews, stars) for all parks.
            """
            try:
                analytics_summary = (
                    db.session.query(
                        Analytics.channel_id,
                        db.func.avg(Analytics.stars).label('stars'),
                        db.func.count(Analytics.id).label('total_reviews')
                    )
                    .group_by(Analytics.channel_id)
                    .all()
                )

                if not analytics_summary:
                    return {'message': 'No analytics data available'}, 404

                return jsonify([
                    {
                        "channel_id": entry.channel_id,
                        "stars": round(entry.stars, 1),
                        "total_reviews": entry.total_reviews
                    }
                    for entry in analytics_summary
                ]), 200
            except Exception as e:
                return {'message': f'An error occurred: {str(e)}'}, 500


